# AutoGen 

AutoGen is an open-source framework that leverages multiple agents to enable complex workflows.

### Agents
In AutoGen, an agent is an entity that can send messages, receive messages and generate a reply using models, tools, human inputs, and from other agents in its environment or a mixture of them.

![alt text](<Screenshot 2024-11-12 191448.png>)

AutoGen offers a robust framework for creating AI-driven systems through various types of agents. Each agent type serves a unique role and can be combined to build complex workflows. In this post, we’ll explore the three main types of agents in AutoGen: ConversableAgent, AssistantAgent, and UserProxyAgent, along with practical examples to illustrate their use.

#### 1. ConversableAgent
ConversableAgents are designed for structured conversations. They facilitate interactions based on predefined dialogue patterns, making them ideal for tasks that require gathering specific information or following a scripted flow.

#### 2. AssistantAgent
AssistantAgents are task-oriented agents that assist with specific tasks, often providing expertise or performing actions based on user requests. They are well-suited for technical support, research assistance, or any task that requires specialized knowledge.

#### 3. UserProxyAgent
UserProxyAgents act as intermediaries between the user and the system. They relay user inputs to the appropriate agents and return responses, ensuring smooth communication across different agents.

In [1]:
pip install autogen

Note: you may need to restart the kernel to use updated packages.


In [13]:
import autogen
from autogen import ConversableAgent, AssistantAgent, UserProxyAgent

In [14]:
#LLM Configuration

llm_config = {
    "timeout":600,
    "config_list": autogen.config_list_from_json("OAI_CONFIG_LIST"),
    "temperature": 0,
}

In [15]:
data_collection_prompt = """ You are a business analyst. Your job is to ask series of questions and collect the requirements from the user. 
Get all the necessary information required for the project from requirement gathering, design, develop, test and deployment.

Don't ask all the questions at once, ask one question at a time for better user experience. 

Once you collect all the information reply 'exit' to pass this information to the next agent"""

In [7]:
data_collection_agent = ConversableAgent(
    name = "data_collection_agent",
    system_message = data_collection_prompt,
    llm_config = llm_config,
    is_termination_msg = lambda msg: "exit" in msg["content"].lower(),
)

In [16]:
estimation_prompt = """
    You are a program manager. Your job is to provide the estimates for the data collected from the data collection agent. You will provide estimates
    with respect to resources required, time taken to complete this project, tech stack required for this process, cost of the tech stack, overall TCO.
"""

In [17]:
estimation_agent = AssistantAgent(
    name = "estimation_agent",
    system_message = estimation_prompt,
    llm_config = llm_config, 
    human_input_mode = "NEVER",
    is_termination_msg = lambda msg: "exit" in msg["content"].lower(),
)

In [18]:
user_proxy_prompt = """" You are an intermediary agent. Your role is to reply user inputs to the appropriate agents and return their responses.
You do not perform any tasks yourself but ensure smooth communication between the user and the system."""

In [19]:
user_proxy = UserProxyAgent(
    name="user_proxy",
    human_input_mode = "ALWAYS",
    llm_config = llm_config,
    code_execution_config = False,
    system_message = user_proxy_prompt,
    is_termination_msg = lambda msg: "exit" in msg["content"].lower(),
)

In [ ]:
chat_results = user_proxy.initiate_chats(
    [
        {
            "sender":data_collection_agent,
            "recipient": user_proxy,
            "message": "I'm a business analyst. I'll gather requirements from you by asking series of questions. Please start by giving a summary of the project which you want to build.",
            "summary_method": "reflection_with_llm",
        },
        {
            "sender":data_collection_agent,
            "recipient": estimation_agent,
            "message": "Here is the estimation for the process collected by the data collection agent.",
            "summary_method": "reflection_with_llm",
        }
    ]
)


********************************************************************************
Starting a new chat....

********************************************************************************
data_collection_agent (to user_proxy):

I'm a business analyst. I'll gather requirements from you by asking series of questions. Please start by giving a summary of the project which you want to build.

--------------------------------------------------------------------------------
user_proxy (to data_collection_agent):

hi

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
data_collection_agent (to user_proxy):

Hello! I'm here to help you gather requirements for your project. Could you please provide a brief summary of the project you want to build?

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

***